In [1]:
import pandas as pd
import json
import os
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium import webdriver
import csv
import re

In [2]:
df = pd.read_csv("scraper_list/google_list_v2.csv")

In [3]:
print(len(df))

19502


In [4]:
df.head()

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link
0,US-11604547-B2,Multipoint touchscreen,Apple Inc.,"Steve Hotelling, Joshua A. Strickon, Brian Q. ...",2004-05-06,2021-01-27,2023-03-14,2023-03-14,https://patents.google.com/patent/US11604547B2/en,https://patentimages.storage.googleapis.com/a1...
1,US-11029838-B2,"Touch screen device, method, and graphical use...",Apple Inc.,"Stephen O. Lemay, Richard Williamson",2006-09-06,2019-12-04,2021-06-08,2021-06-08,https://patents.google.com/patent/US11029838B2/en,https://patentimages.storage.googleapis.com/bf...
2,US-11379541-B2,System and method for adapting a control funct...,Autoconnect Holdings Llc,Christopher P. Ricci,2013-04-15,2021-04-16,2022-07-05,2022-07-05,https://patents.google.com/patent/US11379541B2/en,https://patentimages.storage.googleapis.com/f0...
3,US-11390881-B2,Soybean event MON89788 and methods for detecti...,"Monsanto Technology, Llc","Marianne Malven, Jennifer Rinehart, Nancy Tayl...",2005-05-27,2020-07-01,2022-07-19,2022-07-19,https://patents.google.com/patent/US11390881B2/en,https://patentimages.storage.googleapis.com/3f...
4,US-11423886-B2,Task flow identification based on user intent,Apple Inc.,"Thomas Robert Gruber, Adam John Cheyer, Dag Ki...",2010-01-18,2020-05-20,2022-08-23,2022-08-23,https://patents.google.com/patent/US11423886B2/en,https://patentimages.storage.googleapis.com/6a...


In [5]:
df.columns

Index(['id', 'title', 'assignee', 'inventor/author', 'priority date',
       'filing/creation date', 'publication date', 'grant date', 'result link',
       'representative figure link'],
      dtype='object')

In [6]:
chrome_options = ChromeOptions()
chrome_options.add_argument("--headless")
driver=webdriver.Chrome()

In [7]:
def test_selenium():
    driver.get(df['result link'][1])
    driver.implicitly_wait(5) 
    # pubnum = driver.find_element(By.CSS_SELECTOR, "h2#pubnum")
    # title = driver.find_element(By.CSS_SELECTOR, "h1#title")
    # abstract = driver.find_element(By.CSS_SELECTOR, ".abstract.patent-text")
    # claims_div = driver.find_element(By.CSS_SELECTOR, ".claims.patent-text")
    # claims = claims_div.find_elements(By.CSS_SELECTOR, ".claim-text.patent-text")
    # print(pubnum.text)
    # print(title.text)
    # print(abstract.text)
    print("-----")
    # for claim in claims:
    #     print(claim.text)
    view_more_div = driver.find_element(By.CSS_SELECTOR, "classification-viewer div.more.classification-viewer")
    view_more_div.click()
    classifications = []
    classification_viewer = driver.find_element(By.CSS_SELECTOR, "classification-viewer.patent-result")
    a_tags = classification_viewer.find_elements(By.CSS_SELECTOR, "classification-tree.classification-viewer a")
    for a_tag in a_tags:
        if a_tag.text.strip():
            print(a_tag.text)
            classifications.append(a_tag.text)
    print(classifications)
    print("--------")
#test_selenium()

In [8]:
dataset_folder = "./dataset"
if not os.path.exists(dataset_folder):
    os.makedirs(dataset_folder)
    print("Dataset folder created")
else:
    print("Dataset folder already exists")

Dataset folder already exists


In [9]:
def saveToFile(content):
    file_path = f"./dataset/{content['patent_id']}.json"
    json_data = json.dumps(content, indent=4)
    with open(file_path, 'w') as json_file:
        json_file.write(json_data)
        print(f"File created for {content['patent_id']} at path: {file_path}")

In [10]:
#copy classifications to CSV
global_classifications = []
def save_to_csv():
    global global_classifications
    with open('./scraper_list/classifiction_list.csv', 'a', newline = '') as result_file:
        wr = csv.writer(result_file, dialect='excel')
        for item in global_classifications:
            wr.writerow([item])

In [11]:
def scrape(patent_link, result = {}):
    global global_classifications
    driver = webdriver.Chrome()
    driver.get(patent_link)
    driver.implicitly_wait(5)
    result['patent_link'] = patent_link
    #Inventors and assignee
    people_div = driver.find_element(By.CSS_SELECTOR, ".important-people")
    inventors_st_mod = people_div.find_elements(By.CSS_SELECTOR, "state-modifier a")
    inventors = []
    for inventor in inventors_st_mod:
        inventors.append(inventor.text)
    result['inventors'] = inventors
    
    #classifications
    try:
        view_more_div = driver.find_element(By.CSS_SELECTOR, "classification-viewer div.more.classification-viewer")
        view_more_div.click()
        classifications = []
        classification_viewer = driver.find_element(By.CSS_SELECTOR, "classification-viewer.patent-result")
        a_tags = classification_viewer.find_elements(By.CSS_SELECTOR, "classification-tree.classification-viewer a")
        for a_tag in a_tags:
            if a_tag.text.strip():
                classifications.append(a_tag.text)
        result['classifications'] = classifications
        global_classifications = list(set(global_classifications + classifications))
    except:
        result['classifications'] = []

    try:
        abstract = driver.find_element(By.CSS_SELECTOR, ".abstract.patent-text")
        result['abstract'] = abstract.text
    except:
        result['abstract'] = ""
    try:
        claims_div = driver.find_element(By.CSS_SELECTOR, ".claims.patent-text")
        claims = claims_div.find_elements(By.CSS_SELECTOR, ".claim-text.patent-text")
        claims_text = ""
        for claim in claims:
            if re.match(r"[0-9]+.", claim.text):
                claims_text += "\n"+claim.text
        result['claims'] = claims_text
    except Exception:
        result['claims'] = ""
    return

In [12]:
def parse_csv(df):
    count = 0
    for index,row in df.iterrows():
        if not row['id'].startswith("US"): continue
        print('Scraping document: ', count+1)
        patent_id = row['id']
        content = {}
        content['patent_id'] = patent_id
        content['title'] = row['title']
        content['assignee'] = row['assignee']
        content['publication_date'] = row['publication date']
        patent_link = row['result link']
        print(f"Fetching patent: {patent_id} from {patent_link}")
        dataset_folder = "./dataset"
        if not os.path.exists(dataset_folder+f"/{patent_id}.json"):
            scrape(patent_link, content)
            saveToFile(content)
            save_to_csv()
            print(f"File {dataset_folder}/{patent_id}.json created successfully.")
        else:
            print(f"File {dataset_folder}/{patent_id}.json already exists.")
        count += 1
        if count == 100:
            break
    print(count)

In [ ]:
parse_csv(df)

In [23]:
files = os.listdir("./scraper_list/category_list")
print(len(files), " - total categories")
df = None
for i in range(11, 16):
    df = pd.read_csv(f"./scraper_list/category_list/{files[i]}")
    print(files[i])
    df.columns
    parse_csv(df)

37  - total categories
gp-search-20231104-190320.csv


Index(['id', 'title', 'assignee', 'inventor/author', 'priority date',
       'filing/creation date', 'publication date', 'grant date', 'result link',
       'representative figure link'],
      dtype='object')

In [24]:
parse_csv(df)

Scraping document:  1
Fetching patent: US-11575693-B1 from https://patents.google.com/patent/US11575693B1/en
File ./dataset/US-11575693-B1.json already exists.
Scraping document:  2
Fetching patent: US-11126635-B2 from https://patents.google.com/patent/US11126635B2/en
File created for US-11126635-B2 at path: ./dataset/US-11126635-B2.json
File ./dataset/US-11126635-B2.json created successfully.
Scraping document:  3
Fetching patent: US-11231705-B2 from https://patents.google.com/patent/US11231705B2/en
File ./dataset/US-11231705-B2.json already exists.
Scraping document:  4
Fetching patent: US-10909681-B2 from https://patents.google.com/patent/US10909681B2/en
File created for US-10909681-B2 at path: ./dataset/US-10909681-B2.json
File ./dataset/US-10909681-B2.json created successfully.
Scraping document:  5
Fetching patent: US-11010667-B2 from https://patents.google.com/patent/US11010667B2/en
File created for US-11010667-B2 at path: ./dataset/US-11010667-B2.json
File ./dataset/US-11010667